# Initialization

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from torch.nn.parameter import Parameter
from torchvision.datasets import MNIST
import torch.optim as optim

import torchvision
from torch.autograd import Variable

from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


from torch.nn.parameter import Parameter
from torchvision.datasets import MNIST


In [2]:
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()

    for batch_idx, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(inputs), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, test_loader):
    model.eval()
    total, correct = 0, 0

    for data in test_loader:
        images, labels = data
        outputs = model(Variable(images))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    
    print('Accuracy of the network on the {} test images: {:4.2f} %'.format(
    len(test_loader.dataset), 100 * correct.true_divide(total)))

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 64}

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
device = torch.device("cpu")

# A look into robustness - 02456 Deep Learning project

## Create a high-accuracy network for the MNIST dataset


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv32_1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv32_2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.BN32_1 = nn.BatchNorm2d(32)
        self.BN32_2 = nn.BatchNorm2d(32)
        self.BN32_3 = nn.BatchNorm2d(32)

        self.pool = nn.MaxPool2d(kernel_size=5, stride=2, padding=2)

        self.dropout = nn.Dropout(0.4)

        self.conv64_1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv64_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.BN64_1 = nn.BatchNorm2d(64)
        self.BN64_2 = nn.BatchNorm2d(64)
        self.BN64_3 = nn.BatchNorm2d(64)

        self.conv128 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4)
        self.BN128 = nn.BatchNorm2d(128)

        self.fc = nn.Linear(1*1*128, 10)


    def forward(self, x):                          # (28, 28, 1)
        x = self.BN32_1(F.relu(self.conv32_1(x)))    # (26, 26, 32)
        x = self.BN32_2(F.relu(self.conv32_2(x)))    # (24, 24, 32)
        x = self.BN32_3(F.relu(self.pool(x)))        # (12, 12, 32)

        x = self.dropout(x)

        x = self.BN64_1(F.relu(self.conv64_1(x)))    # (10, 10, 64)
        x = self.BN64_2(F.relu(self.conv64_2(x)))    # (8, 8, 64)
        x = self.BN64_3(F.relu(self.pool(x)))        # (4, 4, 64)

        x = self.dropout(x)

        x = self.BN128(F.relu(self.conv128(x)))     # (1, 1, 128)
        # Flatten
        x = x.view(-1, 1*1*128)

        x = self.dropout(x)

        x = F.softmax(self.fc(x))

        return x

In [ ]:
# Training settings

dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)


In [5]:
model = Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 20 + 1):
    train(model, train_loader, optimizer, criterion, epoch)
    test(model, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn.pt")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.311024
Accuracy of the network on the 10000 test images: 98.77 %
Train Epoch: 2 [0/60000 (0%)]	Loss: 1.476732
Accuracy of the network on the 10000 test images: 99.23 %
Train Epoch: 3 [0/60000 (0%)]	Loss: 1.472849
Accuracy of the network on the 10000 test images: 99.24 %
Train Epoch: 4 [0/60000 (0%)]	Loss: 1.461550
Accuracy of the network on the 10000 test images: 99.40 %
Train Epoch: 5 [0/60000 (0%)]	Loss: 1.476785
Accuracy of the network on the 10000 test images: 99.36 %
Train Epoch: 6 [0/60000 (0%)]	Loss: 1.461397
Accuracy of the network on the 10000 test images: 99.53 %
Train Epoch: 7 [0/60000 (0%)]	Loss: 1.461316
Accuracy of the network on the 10000 test images: 99.52 %
Train Epoch: 8 [0/60000 (0%)]	Loss: 1.461202
Accuracy of the network on the 10000 test images: 99.51 %
Train Epoch: 9 [0/60000 (0%)]	Loss: 1.461267
Accuracy of the network on the 10000 test images: 99.55 %
Train Epoch: 10 [0/60000 (0%)]	Loss: 1.461184
Accuracy of the network on 

# Networks definitions

#### Original

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv32_1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv32_2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.BN32_1 = nn.BatchNorm2d(32)
        self.BN32_2 = nn.BatchNorm2d(32)
        self.BN32_3 = nn.BatchNorm2d(32)

        self.pool = nn.MaxPool2d(kernel_size=5, stride=2, padding=2)

        self.dropout = nn.Dropout(0.4)

        self.conv64_1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv64_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.BN64_1 = nn.BatchNorm2d(64)
        self.BN64_2 = nn.BatchNorm2d(64)
        self.BN64_3 = nn.BatchNorm2d(64)

        self.conv128 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4)
        self.BN128 = nn.BatchNorm2d(128)

        self.fc = nn.Linear(1*1*128, 10)


    def forward(self, x):                          # (28, 28, 1)
        x = self.BN32_1(F.relu(self.conv32_1(x)))    # (26, 26, 32)
        x = self.BN32_2(F.relu(self.conv32_2(x)))    # (24, 24, 32)
        x = self.BN32_3(F.relu(self.pool(x)))        # (12, 12, 32)

        x = self.dropout(x)

        x = self.BN64_1(F.relu(self.conv64_1(x)))    # (10, 10, 64)
        x = self.BN64_2(F.relu(self.conv64_2(x)))    # (8, 8, 64)
        x = self.BN64_3(F.relu(self.pool(x)))        # (4, 4, 64)

        x = self.dropout(x)

        x = self.BN128(F.relu(self.conv128(x)))     # (1, 1, 128)
        # Flatten
        x = x.view(-1, 1*1*128)

        x = self.dropout(x)

        x = F.softmax(self.fc(x))

        return x

#### Avg

In [7]:
class Net_avg(nn.Module):
    def __init__(self):
        super(Net_avg, self).__init__()

        self.conv32_1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv32_2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.BN32_1 = nn.BatchNorm2d(32)
        self.BN32_2 = nn.BatchNorm2d(32)
        self.BN32_3 = nn.BatchNorm2d(32)

        self.pool = nn.AvgPool2d(kernel_size=5, stride=2, padding=2)

        self.dropout = nn.Dropout(0.4)

        self.conv64_1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv64_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.BN64_1 = nn.BatchNorm2d(64)
        self.BN64_2 = nn.BatchNorm2d(64)
        self.BN64_3 = nn.BatchNorm2d(64)

        self.conv128 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4)
        self.BN128 = nn.BatchNorm2d(128)

        self.fc = nn.Linear(1*1*128, 10)


    def forward(self, x):                          # (28, 28, 1)
        x = self.BN32_1(F.relu(self.conv32_1(x)))    # (26, 26, 32)
        x = self.BN32_2(F.relu(self.conv32_2(x)))    # (24, 24, 32)
        x = self.BN32_3(F.relu(self.pool(x)))        # (12, 12, 32)

        x = self.dropout(x)

        x = self.BN64_1(F.relu(self.conv64_1(x)))    # (10, 10, 64)
        x = self.BN64_2(F.relu(self.conv64_2(x)))    # (8, 8, 64)
        x = self.BN64_3(F.relu(self.pool(x)))        # (4, 4, 64)

        x = self.dropout(x)

        x = self.BN128(F.relu(self.conv128(x)))     # (1, 1, 128)
        # Flatten
        x = x.view(-1, 1*1*128)

        x = self.dropout(x)

        x = F.softmax(self.fc(x))

        return x

#### BatchNorm

In [8]:
class Net_BN(nn.Module):
    def __init__(self):
        super(Net_BN, self).__init__()

        self.conv32_1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv32_2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.BN32_1 = nn.BatchNorm2d(32)
        self.BN32_2 = nn.BatchNorm2d(32)
        self.BN32_3 = nn.BatchNorm2d(32)

        self.pool = nn.MaxPool2d(kernel_size=5, stride=2, padding=2)

        self.dropout = nn.Dropout(0.4)

        self.conv64_1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv64_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.BN64_1 = nn.BatchNorm2d(64)
        self.BN64_2 = nn.BatchNorm2d(64)
        self.BN64_3 = nn.BatchNorm2d(64)

        self.conv128 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4)
        self.BN128 = nn.BatchNorm2d(128)

        self.fc = nn.Linear(1*1*128, 10)


    def forward(self, x):                          # (28, 28, 1)
        x = (F.relu(self.conv32_1(x)))    # (26, 26, 32)
        x = (F.relu(self.conv32_2(x)))    # (24, 24, 32)
        x = (F.relu(self.pool(x)))        # (12, 12, 32)

        x = self.dropout(x)

        x = (F.relu(self.conv64_1(x)))    # (10, 10, 64)
        x = (F.relu(self.conv64_2(x)))    # (8, 8, 64)
        x = (F.relu(self.pool(x)))        # (4, 4, 64)

        x = self.dropout(x)

        x = (F.relu(self.conv128(x)))     # (1, 1, 128)
        # Flatten
        x = x.view(-1, 1*1*128)

        x = self.dropout(x)

        x = F.softmax(self.fc(x))

        return x

#### Conv Pool

In [9]:
class Net_ConvPool(nn.Module):
    def __init__(self):
        super(Net_ConvPool, self).__init__()

        self.conv32_1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv32_2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.BN32_1 = nn.BatchNorm2d(32)
        self.BN32_2 = nn.BatchNorm2d(32)
        self.BN32_3 = nn.BatchNorm2d(32)

        # self.pool = nn.AvgPool2d(kernel_size=5, stride=2, padding=2)

        self.pool_1 = nn.Conv2d(in_channels=32, out_channels=32, stride=2, kernel_size=5, padding=2)
        self.pool_2 = nn.Conv2d(in_channels=64, out_channels=64, stride=2, kernel_size=5, padding=2)


        self.dropout = nn.Dropout(0.4)

        self.conv64_1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv64_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.BN64_1 = nn.BatchNorm2d(64)
        self.BN64_2 = nn.BatchNorm2d(64)
        self.BN64_3 = nn.BatchNorm2d(64)

        self.conv128 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4)
        self.BN128 = nn.BatchNorm2d(128)

        self.fc = nn.Linear(1*1*128, 10)


    def forward(self, x):                          # (28, 28, 1)
        x = self.BN32_1(F.relu(self.conv32_1(x)))    # (26, 26, 32)
        x = self.BN32_2(F.relu(self.conv32_2(x)))    # (24, 24, 32)
        x = self.BN32_3(F.relu(self.pool_1(x)))        # (12, 12, 32)

        x = self.dropout(x)

        x = self.BN64_1(F.relu(self.conv64_1(x)))    # (10, 10, 64)
        x = self.BN64_2(F.relu(self.conv64_2(x)))    # (8, 8, 64)
        x = self.BN64_3(F.relu(self.pool_2(x)))        # (4, 4, 64)

        x = self.dropout(x)

        x = self.BN128(F.relu(self.conv128(x)))     # (1, 1, 128)
        # Flatten
        x = x.view(-1, 1*1*128)

        x = self.dropout(x)

        x = F.softmax(self.fc(x))

        return x

#### Dropout

In [10]:
class Net_DO(nn.Module):
    def __init__(self):
        super(Net_DO, self).__init__()

        self.conv32_1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv32_2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.BN32_1 = nn.BatchNorm2d(32)
        self.BN32_2 = nn.BatchNorm2d(32)
        self.BN32_3 = nn.BatchNorm2d(32)

        self.pool = nn.MaxPool2d(kernel_size=5, stride=2, padding=2)

        self.dropout = nn.Dropout(0.4)

        self.conv64_1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv64_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.BN64_1 = nn.BatchNorm2d(64)
        self.BN64_2 = nn.BatchNorm2d(64)
        self.BN64_3 = nn.BatchNorm2d(64)

        self.conv128 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4)
        self.BN128 = nn.BatchNorm2d(128)

        self.fc = nn.Linear(1*1*128, 10)


    def forward(self, x):                          # (28, 28, 1)
        x = self.BN32_1(F.relu(self.conv32_1(x)))    # (26, 26, 32)
        x = self.BN32_2(F.relu(self.conv32_2(x)))    # (24, 24, 32)
        x = self.BN32_3(F.relu(self.pool(x)))        # (12, 12, 32)

        # x = self.dropout(x)

        x = self.BN64_1(F.relu(self.conv64_1(x)))    # (10, 10, 64)
        x = self.BN64_2(F.relu(self.conv64_2(x)))    # (8, 8, 64)
        x = self.BN64_3(F.relu(self.pool(x)))        # (4, 4, 64)

        # x = self.dropout(x)

        x = self.BN128(F.relu(self.conv128(x)))     # (1, 1, 128)
        # Flatten
        x = x.view(-1, 1*1*128)

        x = self.dropout(x)

        x = F.softmax(self.fc(x))

        return x

# AlignMNIST

In [11]:
# Change to the correct path containing the alignmnist.npz file - can be downloaded from github (see readmd)
file = 'drive/MyDrive/alignmnist.npz'

In [12]:
alignmnist = np.load(file)
x = alignmnist['x']
y = alignmnist['y']

x1, y1 = x[:60000], y[:60000]
x2, y2 = x[60000:120000], y[60000:120000]
x3, y3 = x[120000:180000], y[120000:180000]
x4, y4 = x[180000:240000], y[180000:240000]
x5, y5 = x[240000:300000], y[240000:300000]
x6, y6 = x[300000:360000], y[300000:360000]

x1 = [transform(foo) for foo in x1]
x2 = [transform(foo) for foo in x2]
x3 = [transform(foo) for foo in x3]
x4 = [transform(foo) for foo in x4]
x5 = [transform(foo) for foo in x5]
x6 = [transform(foo) for foo in x6]

orig_loader = torch.utils.data.DataLoader(list(zip(x1, y1)), **test_kwargs)
corrupt_loader1 = torch.utils.data.DataLoader(list(zip(x2, y2)), **test_kwargs)
corrupt_loader2 = torch.utils.data.DataLoader(list(zip(x3, y3)), **test_kwargs)
corrupt_loader3 = torch.utils.data.DataLoader(list(zip(x4, y4)), **test_kwargs)
corrupt_loader4 = torch.utils.data.DataLoader(list(zip(x5, y5)), **test_kwargs)
corrupt_loader5 = torch.utils.data.DataLoader(list(zip(x6, y6)), **test_kwargs)

# Train network versions

**Use this section to train the networks yourself (or skip to next for the pre-trained code)**

_In the section after this you will find the code for the pre-trained models, that are downloadable from github_

## BatchNorm

In [ ]:
device = torch.device("cpu")
model = Net_BN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 20 + 1):
    train(model, train_loader, optimizer, criterion, epoch)
    test(model, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn_bn.pt")
print()
print()
test(model, orig_loader)
test(model, corrupt_loader1)

## Dropout

In [ ]:
device = torch.device("cpu")
model = Net_DO().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 20 + 1):
    train(model, train_loader, optimizer, criterion, epoch)
    test(model, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn_do.pt")

print()
print()
test(model, orig_loader)
test(model, corrupt_loader1)

## Average pool

In [ ]:
device = torch.device("cpu")
model = Net_avg().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 20 + 1):
    train(model, train_loader, optimizer, criterion, epoch)
    test(model, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn_avg-1.pt")
print()
print()
test(model, orig_loader)
test(model, corrupt_loader1)

## ConvPool

In [ ]:
device = torch.device("cpu")
model = Net_pool().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 20 + 1):
    train(model, train_loader, optimizer, criterion, epoch)
    test(model, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn_pool.pt")
print()
print()
test(model, orig_loader)
test(model, corrupt_loader1)

## StepLR

In [ ]:
device = torch.device("cpu")
model = Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 20 + 1):
    train(model, train_loader, optimizer, criterion, epoch)
    test(model, test_loader)
    #scheduler.step()

torch.save(model.state_dict(), "mnist_cnn_steplr.pt")
print()
print()
test(model, orig_loader)
test(model, corrupt_loader1)

# Pre-trained

**In order to use this, download the nets folder from github and upload to the current runtime. It contains the pre-trained networks**

_If the networks are manually trained, using the section above, each network must be put inside a folder named 'nets', for the below code to run_

#### Original

In [13]:
model = Net().to(device)
model.load_state_dict(torch.load('nets/mnist_cnn.pt'))
model.eval()
test(model, orig_loader)
test(model, corrupt_loader1)
test(model, corrupt_loader2)
test(model, corrupt_loader3)
test(model, corrupt_loader4)
test(model, corrupt_loader5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 60000 test images: 99.76 %
Accuracy of the network on the 60000 test images: 98.73 %
Accuracy of the network on the 60000 test images: 98.72 %
Accuracy of the network on the 60000 test images: 98.69 %
Accuracy of the network on the 60000 test images: 98.76 %
Accuracy of the network on the 60000 test images: 98.77 %


#### AveragePool

In [14]:
model = Net_avg().to(device)
model.load_state_dict(torch.load('nets/mnist_cnn_avg.pt'))
model.eval()

test(model, orig_loader)
test(model, corrupt_loader1)
test(model, corrupt_loader2)
test(model, corrupt_loader3)
test(model, corrupt_loader4)
test(model, corrupt_loader5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 60000 test images: 99.70 %
Accuracy of the network on the 60000 test images: 98.45 %
Accuracy of the network on the 60000 test images: 98.44 %
Accuracy of the network on the 60000 test images: 98.39 %
Accuracy of the network on the 60000 test images: 98.45 %
Accuracy of the network on the 60000 test images: 98.43 %


#### BatchNorm

In [15]:
model = Net_BN().to(device)
model.load_state_dict(torch.load('nets/mnist_cnn_bn.pt'))
model.eval()

test(model, orig_loader)
test(model, corrupt_loader1)
test(model, corrupt_loader2)
test(model, corrupt_loader3)
test(model, corrupt_loader4)
test(model, corrupt_loader5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 60000 test images: 99.25 %
Accuracy of the network on the 60000 test images: 98.03 %
Accuracy of the network on the 60000 test images: 97.96 %
Accuracy of the network on the 60000 test images: 97.98 %
Accuracy of the network on the 60000 test images: 98.02 %
Accuracy of the network on the 60000 test images: 97.91 %


#### StepLR

In [16]:
model = Net().to(device)
model.load_state_dict(torch.load('nets/mnist_cnn_steplr.pt'))
model.eval()

test(model, orig_loader)
test(model, corrupt_loader1)
test(model, corrupt_loader2)
test(model, corrupt_loader3)
test(model, corrupt_loader4)
test(model, corrupt_loader5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 60000 test images: 99.71 %
Accuracy of the network on the 60000 test images: 98.58 %
Accuracy of the network on the 60000 test images: 98.66 %
Accuracy of the network on the 60000 test images: 98.58 %
Accuracy of the network on the 60000 test images: 98.64 %
Accuracy of the network on the 60000 test images: 98.57 %


#### ConvPool

In [17]:
model = Net_ConvPool().to(device)
model.load_state_dict(torch.load('nets/mnist_cnn_pool.pt'))
model.eval()

test(model, orig_loader)
test(model, corrupt_loader1)
test(model, corrupt_loader2)
test(model, corrupt_loader3)
test(model, corrupt_loader4)
test(model, corrupt_loader5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 60000 test images: 99.78 %
Accuracy of the network on the 60000 test images: 98.29 %
Accuracy of the network on the 60000 test images: 98.21 %
Accuracy of the network on the 60000 test images: 98.26 %
Accuracy of the network on the 60000 test images: 98.29 %
Accuracy of the network on the 60000 test images: 98.23 %


#### Dropout

In [18]:
model = Net_DO().to(device)
model.load_state_dict(torch.load('nets/mnist_cnn_do.pt'))
model.eval()

test(model, orig_loader)
test(model, corrupt_loader1)
test(model, corrupt_loader2)
test(model, corrupt_loader3)
test(model, corrupt_loader4)
test(model, corrupt_loader5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 60000 test images: 99.88 %
Accuracy of the network on the 60000 test images: 98.57 %
Accuracy of the network on the 60000 test images: 98.54 %
Accuracy of the network on the 60000 test images: 98.58 %
Accuracy of the network on the 60000 test images: 98.61 %
Accuracy of the network on the 60000 test images: 98.53 %
